# 6.5 循环神经网络的简洁实现

In [1]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

print(torch.__version__, device)


1.3.1 cpu


## 6.5.1 定义模型

In [2]:
num_hiddens = 256
# rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens) # 已测试
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)#这是pytorch里的RNN类
                                            #由于用的是one-hot编码，所以需要词库个数纬度的输入
    

In [4]:
num_steps = 35
batch_size = 2
state = None
print(vocab_size)# 表示周杰伦歌词里不重复的字的个数，这里由于d2l.load_data_jay_lyrics()函数里只用前10000个字符
                 # 来计算，原本总的字符是60000，不重复的字符个数为2084个
X = torch.rand(num_steps, batch_size, vocab_size)# torch.rand是均匀分布生成随机数，
                                                 # torch.randn是正态分布生成随机数
Y, state_new = rnn_layer(X, state)
print(X.shape, X)
print(Y.shape, len(state_new), state_new[0].shape)

1027
torch.Size([35, 2, 1027]) tensor([[[0.2232, 0.2926, 0.9915,  ..., 0.5634, 0.7594, 0.7052],
         [0.3929, 0.5466, 0.4986,  ..., 0.2145, 0.0232, 0.7676]],

        [[0.4312, 0.2995, 0.4645,  ..., 0.0458, 0.5486, 0.0252],
         [0.7800, 0.3216, 0.0185,  ..., 0.3905, 0.6861, 0.9121]],

        [[0.4057, 0.2213, 0.1455,  ..., 0.2836, 0.7666, 0.5708],
         [0.4670, 0.6297, 0.7825,  ..., 0.8081, 0.7983, 0.7507]],

        ...,

        [[0.6840, 0.8226, 0.7421,  ..., 0.2564, 0.0068, 0.0544],
         [0.4239, 0.4592, 0.3380,  ..., 0.1409, 0.5305, 0.3794]],

        [[0.7578, 0.0346, 0.0217,  ..., 0.0909, 0.2422, 0.5392],
         [0.6214, 0.6729, 0.5669,  ..., 0.9440, 0.5486, 0.9684]],

        [[0.9621, 0.4905, 0.5012,  ..., 0.5769, 0.2105, 0.9054],
         [0.8896, 0.3834, 0.5496,  ..., 0.2841, 0.8867, 0.6423]]])
torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [ ]:
# 本类已保存在d2lzh_pytorch包中方便以后使用
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)#输出层的全连接
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X = d2l.to_onehot(inputs, vocab_size) # X是个list
        Y, self.state = self.rnn(torch.stack(X), state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

## 6.5.2 训练模型

In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)
            
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [ ]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()
    
            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            # 梯度裁剪
            d2l.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [ ]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)